# Project Report - Groupe 28
**Authors :** Celest Angela Tjong, Adrien Louis Baptiste Dupont, Luca Sidoti Pinto, Didier Henri Neuenschwander

**Supervisors :** Prof. Francesco Mondada

**Date** : 7/12/2023

**[MICRO-452] : Basics of mobile robotics**

---

# Table of Contents
* [1. Introduction](#introduction)
* [2. Initialisation](#initialisation)
* [3. Vision](#vision)
    * [3.1. Subsection 1](#vision-subsection-1)
    * [3.2. Subsection 2](#vision-subsection-2)
* [4. Global Navigation](#global-navigation)
    * [4.1 Path planning](#path-planning)
    * [4.2 A* implementation](#A*)
* [5. Filtering](#filtering)
    * [5.1 State-space model](#state-space-model)
    * [5.2 Kalman filter](#kalman-filter)
* [6. Local Navigation](#local-navigation)
* [7. Motion control](#motion-control)
    * [7.1 Connection functions](#connection-functions)
    * [7.2 Motion functions](#motion-functions)
    * [7.3 Motion control function](#motion-control-function)
* [8. Projet structure](#project-structure)   
* [9. Runnable cells](#runnable-cells)
    * [9.1 Functions declaration ](#functions-declaration)
    * [9.2 Capture image](#capture-image)
    * [9.3 Main](#main)
* [10. Conclusion](#conclusion)

---
<a id="introduction"></a>

# Introduction

This project, integrated into the MICRO-452 Basics of Mobile Robotics course, aims to combine various essential elements of mobile robotics: vision, global and local navigation, motion control, and filtering.

Here is how the project was implemented. Initially, a webcam captures the image of the environment. The necessary information for operation, such as the robot’s position, the map, static obstacles, and the target’s position, are extracted in real-time using conventional image processing techniques. Subsequently, the A* algorithm is used to determine the optimal path. The robot follows this path, while the Kalman filter estimates the position during movement. Knowledge of the path and the robot’s position allows for control of its movement. If the Thymio detects an obstacle using its horizontal proximity sensors, local navigation takes over to avoid a collision. In addition, the robot can find its way back to the target in case of kidnapping. Finally, the Thymio is designed to reach the target even if the camera’s vision is obstructed.

---
<a id="initialisation"></a>

# Initialisation

In [6]:
# IMPORTS :


import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors
from PIL import Image
import math
import time
from ipywidgets import interactive
%matplotlib inline
import asyncio
from tdmclient import aw, ClientAsync


# GLOBAL CONSTANTS :

reduction_coeff = 25 # A METTRE EN MAJUSCULE
ROTATION_COST = 1
ROTATION_MODE = 0
FORWARD_MODE = 1
ANGLE_THRESHOLD = 0.0
FORWARD_THRESHOLD = 1
ROTATION_TIME_THRESHOLD = 1.2
FORWARD_TIME_THRESHOLD = 100
TURN_RIGHT=0
TURN_LEFT=1
ROTATION_SPEED = 100
TIME_FULL_TURN = (8800/1000)

# A PASSER EN CAPITALES
reduction_coeff = 25
Thymio_to_mms = 0.349
px_to_mm = 137/100
## Parameters for local navigation
pi = math.pi
angle_turned = 0
distance_forward = 1
obstThrh = 20     # high obstacle threshold to switch state from global nav to local

# Define color thresholds in HSV
lower_red_bound = np.array([120, 100, 70])
upper_red_bound = np.array([255, 255, 255])
lower_green_bound = np.array([60, 50, 100])
upper_green_bound = np.array([100, 255, 255])
lower_yellow_bound = np.array([0, 50, 120])
upper_yellow_bound = np.array([40, 105, 255])
lower_black_bound = np.array([0, 0, 0])
upper_black_bound = np.array([255, 255, 130])
lower_blue_bound = np.array([90, 80, 0])
upper_blue_bound = np.array([105, 255, 255])





import cv2


TypeError: 'numpy._DTypeMeta' object is not subscriptable

---
<a id="vision"></a>
## Vision

https://docs.opencv.org/4.x/df/d9d/tutorial_py_colorspaces.html


The first think to do is to preprocess the image for the colour object needed to be located. It is done by first creating a mask. The mask is created by specifying a range of colors (in HSV color space). Pixels within this color range are marked as 1 (or true), while all other pixels are marked as 0 (or false). it is created using `cv2.inRange` which filters out all colors except those within the specified `lower_color_bound` and `upper_color_bound`. This step isolated the specified color. 
Then the function `cv2.bitwise_and`, extract the area corresponding with the range of colour of the image given as input. It is done by comaparing each pixel of the image with the mask (same size of the image, comaparing with a logical &). 
Finally The color-filtered image is converted to grayscale using `cv2.cvtColor` because the subsequent edge detection step (Canny) requires a single-channel image. 
In order to detect the different coloured form, it is common to begin by using a Canny filter.

### Function: `detect_color_circle`

#### Purpose:
The `detect_color_circle` function is designed to detect circles of a specific color in an image. It employs color filtering, Canny edge detection, and the Hough Circle Transform to achieve this.

#### Process:



2. **Color Masking**:
   - A mask is created using `cv2.inRange` which filters out all colors except those within the specified `lower_color_bound` and `upper_color_bound`. This step isolates the regions of the specified color.

3. **Mask Application**:
   - The mask is then applied to the original image using `cv2.bitwise_and`. This step ensures that only the parts of the image with the desired color are retained for further processing.

4. **Grayscale Conversion**:
   - The color-filtered image is converted to grayscale using `cv2.cvtColor` because the subsequent edge detection step (Canny) requires a single-channel image.

5. **Canny Edge Detection**:
   - `cv2.Canny` is applied to detect edges in the image. It works by identifying areas in the image where sharp changes in intensity occur. The function takes two threshold values (here, 100 and 200) that determine the sensitivity of the edge detection. Edges that are found are used as input for the circle detection.

6. **Hough Circle Transform**:
   - `cv2.HoughCircles` is used to detect circles in the image. It operates on the principle of the Hough Transform, which is a feature extraction technique used in image analysis. The function detects circles by finding sets of edge points that form a circular shape.
   - Parameters like `param1` (higher threshold of the two passed to the Canny edge detector), `param2` (threshold for center detection in the Hough Transform), `minRadius`, and `maxRadius` control the sensitivity and size of the circles to be detected.

7. **Output**:
   - If circles are detected, the function returns a list of tuples, each containing the `(x, y)` coordinates of the center of a circle and its radius. If no circles are found, it returns an empty list.




In [ ]:
# Define a function to detect circles of a specific color
def detect_color_circle(image, lower_color_bound, upper_color_bound):
    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Create a mask for the specified color
    mask = cv2.inRange(hsv, lower_color_bound, upper_color_bound)

    # Apply the mask to the original image
    color_only = cv2.bitwise_and(image, image, mask=mask)

    # Convert to grayscale for circle detection
    gray = cv2.cvtColor(color_only, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection to help with circle detection
    edges = cv2.Canny(gray, 100, 200)

    # Use Hough Transform to detect circles
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 20,
                           param1=20, param2=15, minRadius=10, maxRadius=50)
    
    # If circles are detected, return the list of circles with x, y coordinates and radius
    if circles is not None:
        # Convert the (1, N, 3) array to (N, 3)
        circles = np.uint16(np.around(circles[0, :]))          
        return [(circle[0], circle[1], circle[2]) for circle in circles]
    else:
        return []
    
#--------------------------------------------------------------------------------#
    
def create_obstacle_mask(image, contours, kernel_size):
    """
    Create a mask with zeros in the areas inside the dilated contours.

    :param image: Input image.
    :param contours: Contours to dilate and fill in the mask.
    :param kernel_size: Size of the kernel used for dilation.
    :return: Mask with zeros inside the dilated contours and ones elsewhere.
    """
    # Create an empty mask of the same size as the image
    h, w = image.shape[:2]
    mask = np.ones((h, w), dtype=np.uint8)

    # Perform dilation to increase the size of the black regions
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    for contour in contours:
        # Create an individual mask for each contour
        contour_mask = np.zeros((h, w), dtype=np.uint8)
        cv2.fillPoly(contour_mask, [contour], 255)
        contour_mask = cv2.dilate(contour_mask, kernel, iterations=1)
        
        # Combine the individual mask with the global mask
        mask = cv2.bitwise_and(mask, cv2.bitwise_not(contour_mask))

        #also add the contours
        # Let's create a border around the image
        border_size = 50
        border_color = [0, 0, 0]  # Black border
        # Use cv2.copyMakeBorder to add a border around the image
        mask_with_border = cv2.copyMakeBorder(mask, border_size, border_size, border_size, border_size,
                                           cv2.BORDER_CONSTANT, value=border_color)
    
    return mask_with_border


# Now you have a mask with zeros in the obstacle areas and ones elsewhere
# You can return this mask from your function or process it further as needed

#--------------------------------------------------------------------------------#

def detect_obstacle_contours(image, area_threshold, kernel_size):
    """
    Detects and dilates obstacle contours in the given image.
    :param image: Input image.
    :param area_threshold: Area threshold for filtering contours.
    :param kernel_size: Size of the kernel used for dilation.
    :return: Image with obstacle contours drawn.
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask_black = cv2.inRange(hsv, lower_black_bound, upper_black_bound)
    contours, _ = cv2.findContours(mask_black, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > area_threshold]
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    mask_dilated = cv2.dilate(mask_black, kernel, iterations=1)
    dilated_contours, _ = cv2.findContours(mask_dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    filtered_dilated_contours = [cnt for cnt in dilated_contours if cv2.contourArea(cnt) > area_threshold+10000]
    contour_image = image.copy()
    cv2.drawContours(contour_image, filtered_contours, -1, (0, 255, 0), 2)
    cv2.drawContours(contour_image, filtered_dilated_contours, -1, (0, 0, 255), 2)
    return contour_image, filtered_contours, filtered_dilated_contours

#--------------------------------------------------------------------------------#

def create_obstacle_matrix(image, dilated_contours):
    height, width = image.shape[:2]
    obstacle_matrix = np.ones((height, width), dtype=np.uint8)

    for contour in dilated_contours:
        # Remplir chaque contour dilaté avec 0 (obstacle)
        cv2.fillPoly(obstacle_matrix, [contour], 0)

    return obstacle_matrix

#--------------------------------------------------------------------------------#

def crop_roi_from_circles(image, circles):
    if circles is not None and len(circles) >= 4:
        # Assurez-vous que les points sont dans le format correct
        points = np.array([circle[:2] for circle in circles], dtype=np.float32)

        # Calcul de la boîte englobante
        rect = cv2.boundingRect(points)

        # Recadrage de l'image
        x, y, w, h = rect
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image,(x, y, w, h)
    else:
        print("Nombre insuffisant de cercles détectés ou format incorrect.")
        return image,None 

#--------------------------------------------------------------------------------#

def find_rectangle_center(image, lower_colour_bound, upper_colour_bound, area_threshold=1000):
    # Convertir l'image en espace de couleur HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Créer un masque pour la couleur 
    mask = cv2.inRange(hsv, lower_colour_bound, upper_colour_bound)

    # Trouver les contours dans le masque
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Identifier le contour qui correspond au rectangle 
    for contour in contours:
        if cv2.contourArea(contour) > area_threshold:
            # Calculer la boîte englobante pour le contour
            x, y, w, h = cv2.boundingRect(contour)

            # Effacer le rectangle en le dessinant en blanc
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 255), -1)

            # Calculer le centre du rectangle
            center = (x + w // 2, y + h // 2)
            return center

    return None

In each frame, there are multiple elements that need to be displayed. The first category includes static elements such as the contours of obstacles and the goal point. Since these do not change over time, it is computationally more efficient to identify and locate them in the first frame, and then display them consistently in subsequent frames.

On the other hand, elements related to the robot's localization must be determined in each frame. To gather information about its orientation and location, three circles are placed on the top of the robot. A green circle is located at the middle back, and two red circles are positioned on each side, with their centers aligned with the robot's ceeris. This arrangement facilitates the extraction of necessary information. By connecting the centers of the two red circles and creating a midpoint, a vector c thenan be formed by connecting this midpoint to the center of the green circ.It might also be feasible to use only two markers (circles), one at the back and one at the front, distinguished by their color. However, the marker at the front would hide the press button.

### Graphe parallaxe

In the context of our robotics project, we encountered a specific challenge related to detecting our robot's position. When the robot was perfectly positioned on a specific target, a sheet placed on the ground, it did not report the expected coordinates. We identified that this problem was due to parallax caused by the thickness of the robot. Due to its viewing angle, the aerial camera distorted the real position of the robot. To solve this problem, we projected the robot's position onto a 2D plane and used correction functions to adjust the coordinates. This allowed us to read the robot's position more accurately : 

For the x axis, we computated an equation fitting our measures : 

$$ f_{corr,x}(x) = 0.96x + 16.67$$

![interactions](Images/graphe_1.jpg)

For the y axis : 

$$ f_{corr,y}(x) = 0.95x + 14.29$$

![interactions](Images/graphe_2.jpg)

In [ ]:
def robot_info(frame):
    global camera_on
    red_circles = detect_color_circle(frame, lower_red_bound, upper_red_bound)
    green_circles = detect_color_circle(frame, lower_green_bound, upper_green_bound)
    
    if red_circles and green_circles and len(red_circles) >= 2:
        # Calculate robot's position and orientation vector
        # Calculate the midpoint between the centers of the red circles
        midpoint = ((red_circles[0][0] + red_circles[1][0]) // 2,
        (red_circles[0][1] + red_circles[1][1]) // 2)
        # Calculate the directional vector
        direction = np.array([midpoint[0] - green_circles[0][0], midpoint[1] - green_circles[0][1]])

        # Normalize and extend the vector
        length = 30  # Additional length
        direction = direction / np.linalg.norm(direction) * length

        # Calculate the new endpoint
        new_endpoint = (int(green_circles[0][0] + direction[0]), int(green_circles[0][1] + direction[1]))

        # Draw the extended arrow
        cv2.arrowedLine(frame, new_endpoint, green_circles[0][:2], (0, 0, 0), 3)

        # Calculate the angle of orientation with respect to the x-axis
        dx =   midpoint[0] - green_circles[0][0]
        dy =   midpoint[1] - green_circles[0][1]
        angle = math.atan2(dy, dx)
        angle_degrees = math.degrees(angle)
        if(angle_degrees>= 0):
            angle_degrees =180 - angle_degrees
        elif (angle_degrees < 0):
            angle_degrees = -(180 + angle_degrees)
        robot_vector = (midpoint[0], midpoint[1], np.radians(angle_degrees)) #information of the robot
        #print("ANGLE ",robot_vector[2])
        # Optionally, display the angle
        cv2.putText(frame, f'Angle: {angle_degrees:.2f} degrees', (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.putText(frame, f'Midpoint: ({midpoint[0]}, {midpoint[1]})', (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.circle(frame, midpoint,10, (255, 0, 0), -1)
       # cv2.circle(frame, (x, y), r, (0, 255, 255), 3)
        camera_on = True
        return midpoint[0], midpoint[1], angle_degrees#information of the robot

    else:
        camera_on = False
        print("PASSAGE A FALSE")
        return None,None,None
    
#--------------------------------------------------------------------------------#
    
def preprocess_image(video_capture, lower_blue_bound, upper_blue_bound):
    ret, initial_frame = video_capture.read()
    if ret:
        #initial_frame, cropping_coords = crop_largest_white_area(initial_frame, 200000)
        blue_circles = detect_color_circle(initial_frame, lower_blue_bound, upper_blue_bound)
        initial_frame ,cropping_coords= crop_roi_from_circles(initial_frame, blue_circles)
        goal_center = find_rectangle_center(initial_frame, lower_blue_bound, upper_blue_bound,2000)
        if goal_center:
            # Store the coordinates of the detected yellow circle
            yellow_circle_coords = goal_center  # center est déjà un tuple (x, y)
            radius = 10  # Rayon du cercle, vous pouvez ajuster cette valeur
            color = (0, 255, 255)  # Couleur jaune en BGR
            cv2.circle(initial_frame, yellow_circle_coords, radius, color, 3)
        robot_vector = robot_info(initial_frame)
        if robot_vector[0] and robot_vector[1]:
            cv2.circle(initial_frame, (robot_vector[0], robot_vector[1]),80, (255, 255, 255), -1)    
        contour_image = detect_obstacle_contours(initial_frame, 2000, 80)
        global_obstacle = create_obstacle_matrix(initial_frame,contour_image[2])
        # Dessiner le cercle
        initial_frame = np.flipud(initial_frame)
        plt.imshow(cv2.cvtColor(contour_image[0], cv2.COLOR_BGR2RGB))
        plt.title('Initial Contours')
        plt.axis('off')
        plt.show()
        return initial_frame, cropping_coords, contour_image, global_obstacle,goal_center, robot_vector
    else:
        return None, None, None, None, None, None
    
#--------------------------------------------------------------------------------#

def process_image(frame, cropping_coords, contour_image, update_rate):
    if cropping_coords is not None:
        x, y, w, h = cropping_coords
        frame = frame[y:y+h, x:x+w]
    # Detect red and green circles
    robot_vector = robot_info(frame)
    if  robot_vector is not None:
        cv2.drawContours(frame, contour_image[1], -1, (0, 255, 0), 2)
        cv2.drawContours(frame, contour_image[2], -1, (0, 0, 255), 2)

        # Display the resulting frame
        cv2.imshow('Robot Detection', frame)
    return robot_vector

#--------------------------------------------------------------------------------#

def x_robot_projection_to_ground(x_pos_robot):
    return round(0.96*x_pos_robot+16.67)

#--------------------------------------------------------------------------------#

def y_robot_projection_to_ground(y_pos_robot):
    return round(0.95*y_pos_robot+14.29) 


In [ ]:
# A DEGAGER AUTRE PART

video_capture = cv2.VideoCapture(1, cv2.CAP_DSHOW)

video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1080)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
window_name = 'Robot Detection'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

# Resize the window (width, height)
cv2.resizeWindow(window_name, 540, 360)  

# Initial detection of obstacles and goal
initial_frame, cropping_coords, contour_image, global_obstacle,goal_center, robot_pose = preprocess_image(video_capture, lower_blue_bound, upper_blue_bound)
# Robot update frequency (10 Hz)
update_rate = 0.01  # 10 times per second
if initial_frame is not None:
    try:
        while True:
            start_time = time.time()
            ret, frame = video_capture.read()
            if not ret:
                break
            robot_pose = process_image(frame, cropping_coords, contour_image, update_rate)

            # Pause to maintain the update frequency
            time_to_wait = max(int((start_time + update_rate - time.time()) * 1000), 1)
            if cv2.waitKey(time_to_wait) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Release the capture when everything is finished
        video_capture.release()
        cv2.destroyAllWindows()

In [ ]:
# A DEGAGER AUTRE PART

#print(goal_center)
#print(robot_pose)
robot_vector = np.zeros(3)
robot_vector[0] = int(x_robot_projection_to_ground(robot_pose[0]))
robot_vector[1] = int(y_robot_projection_to_ground(robot_pose[1]))
robot_vector[2] = np.radians(robot_pose[2])
print(robot_vector)

---
<a id="global-navigation"></a>
# Global navigation

<a id="path-planning"></a>
## Path planning

The path search is transformed into a graph search. A problem encountered is the computational time needed for a grid the size of the image. To reduce this duration a size reduction coefficient is used to downsize the original grid. The value of this coefficient is chosen to have the best trade-off between computational time and precision.

| `reduction_coeff` | Computation time |
|----------|----------|
| 10 | 9.599 |
| 15 | 2.298 |
| 20 | 0.513 |
| 25 | 0.193 |
| 30 | 0.099 |

The solution with the best trade-off is a reduction factor of 25, allowing an average computational time of 0.193s while keeping acceptable precision.

In [ ]:
def  create_downsized_grid(global_obstacle):
    """ 
    Creation of the downsized grid occupation_grid with a scale of reduction_coeff
    
    param : 
        - global_obstacle : grid containing the obstacle positions with original size
    return : 
        - occupancy_grid: Downsized grid with averaged obstacle positions
    """

    # Invert the data to have a value of 0 for free cells and 1 for occupied cells.
    # global_obstacle = np.logical_not( global_obstacle )  
    max_val_x_init = global_obstacle.shape[0]
    max_val_y_init = global_obstacle.shape[1]

    # The size of the new grid is calculated from the size of the original grid global_obstacle
    max_val_x = int(max_val_x_init / reduction_coeff)
    max_val_y = int(max_val_y_init / reduction_coeff)

    # Creation of the downsized grid with
    occupancy_grid = np.zeros((max_val_x, max_val_y), dtype=int)

    # Inspection of squares (of size reduction_coeff*reduction_coeff) in the original grid
    for i in range (max_val_x):
        for j in range (max_val_y):
            sum_pixels = 0

            for k in range (reduction_coeff):

                # Verification that the indexs indice_x and indice_y are in the grid range
                indice_x = int(i * reduction_coeff - reduction_coeff/2 + k)
                if (indice_x < 0):
                    indice_x = 0
                elif (indice_x > (max_val_x_init - 1)):
                    indice_x = max_val_x_init -1

                indice_y = int(j * reduction_coeff - reduction_coeff/2 + k)
                if (indice_y < 0):

                    indice_y = 0
                elif (indice_y > (max_val_y_init - 1)):
                    indice_y = max_val_y_init -1

                # The sum of all values present in the inspected square is stored in sum_pixels
                sum_pixels = sum_pixels + global_obstacle[indice_x][indice_y]


            # If an obstacle is found in the square, a 1 is placed in the corresponding position in the new grid
            if sum_pixels == 0:
                occupancy_grid[i][j] = 0
            else:
                occupancy_grid[i][j] = 1

    return occupancy_grid

#--------------------------------------------------------------------------------#

def create_empty_plot(max_val_x, max_val_y):
    """
    Creation of a figure to plot a matrix of max size max_val 
    
    param : 
        - max_val_x : maximum dimension of the matrix on x axis
        - max_val_y : maximum dimension of the matrix on y axis
    return:
        -  fig : general figure
        -  ax : axis information
    """
    
    fig, ax = plt.subplots(figsize=(7,7))
    
    major_ticks = np.arange(0, max_val_y+1, 5)
    minor_ticks = np.arange(0, max_val_y+1, 1)
    ax.set_xticks(major_ticks)
    ax.set_xticks(minor_ticks, minor=True)
    ax.set_yticks(major_ticks)
    ax.set_yticks(minor_ticks, minor=True)
    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    
    # Definition of axis limits with the information of the matrix size
    ax.set_ylim([max_val_x,-1])
    ax.set_xlim([-1,max_val_y])
    ax.grid(True)
    
    return fig, ax


<a id="path-planning"></a>
## A* implementation

The path planning for the Thymio is made using a hybrid A* algorithm. This ensures an optimised search by using the distance to goal to choose the nodes to explore. The formula used is the following : 

$$ f(n) = g(n) + h(n) $$

Here f(n) is the function we try to optimise, with g(n) being the motion cost and h(n) the heuristic function. The motion cost has here been tailored to the Thymio's mechanism, as it takes into account on one side the cost of forward motion with `deltacost` and on the other side the rotational cost with `rotational_cost`. Adding the rotational cost is key to avoid repetitive direction changes.

The `path_final` matrix is structured such that the y-coordinates are on the first row and the x-coordinates on the second. Consequently, an inversion of these two rows is required prior to executing the robot's movement.

Moreover, the downsizing operation of the original grid introduces inaccuracies regarding the initial and final position of the robot. To mitigate this limitation, the exact coordinates of these two specific positions are preferred over the values returned by the A* algorithm when constructing the movement array.

In [ ]:
def _get_movements_8n():
    """
    Get all possible 8-connectivity movements : up, down, left, right and the 4 diagonals.
    
    return : 
        - list of movements with cost [(dx, dy, movement_cost)]
    """
    s2 = math.sqrt(2)
    return [(1, 0, 1.0),
            (0, 1, 1.0),
            (-1, 0, 1.0),
            (0, -1, 1.0),
            (1, 1, s2),
            (-1, 1, s2),
            (-1, -1, s2),
            (1, -1, s2)]

#--------------------------------------------------------------------------------#

def reconstruct_path(cameFrom, current):
    """
    Reconstructs the path from start node to the current node
    
    param :
        - cameFrom: map (dict) containing for each node the node immediately 
                     preceding it on the cheapest path currently known.
        - param current : current node (x, y)
        
    return : 
        - total_path : list of nodes from start to current node
    """
    
    # The first item is the current node
    total_path = [current]
    
    while current in cameFrom.keys():
        # Add the previous node to the start of the list
        total_path.insert(0, cameFrom[current]) 
        current = cameFrom[current]
    return total_path

#--------------------------------------------------------------------------------#

def A_Star(start, goal, h, coords, occupancy_grid):
    """
    Implementation of the A* algorithm, finding the cheapest path from start to goal 
    
    param :
        - start : start node (x, y)
        - goal: goal node (x, y)
        - h : heuristic function (distance to goal)
        - coords : coordinates in the grid
        - occupancy_grid: downsized grid map with obstacle information
    
    return : 
        - a tuple containing the path distance and the path information in an array of indexs
    """
    
    # Size of the downscaled grid
    max_val_x = occupancy_grid.shape[0]
    max_val_y = occupancy_grid.shape[1]
    
    # Check if the start and goal are within the limits of the grid's size
    for point in [start, goal]:
        for coord in point:
            assert coord>=0 and coord<max_val_y, "start or end goal not contained in the map"
    
    # Check if start and goal nodes are situated on an obstacle
    if occupancy_grid[start[0], start[1]]:
        raise Exception('Start node is not traversable')

    if occupancy_grid[goal[0], goal[1]]:
        raise Exception('Goal node is not traversable')
    
    # Information on the possible movements
    movements = _get_movements_8n()
    
    # --------------------------------------------------------------#
    #                  A* ALGORITHM IMPLEMENTATION                  #
    # --------------------------------------------------------------#
    
    # The set of visited nodes, which are the starting points for neighbor exploration
    # The start node is the only one known yet.
    openSet = [start]
    
    # The set of visited nodes that no longer need to be expanded.
    closedSet = []

    # map containing for each node the node immediately preceding it on the cheapest path currently known.
    cameFrom = dict()

    # For node n, gScore[n] is the cost of the cheapest path from start to n currently known.
    gScore = dict(zip(coords, [np.inf for x in range(len(coords))]))
    gScore[start] = 0

    # For node n, fScore[n] := gScore[n] + h(n). map with default value of Infinity
    fScore = dict(zip(coords, [np.inf for x in range(len(coords))]))
    fScore[start] = h[start]
    

    # While there are still cells to explore
    while openSet != []:
        
        # The node in openSet having the lowest fScore[] value
        fScore_openSet = {key:val for (key,val) in fScore.items() if key in openSet}
        current = min(fScore_openSet, key=fScore_openSet.get)
        del fScore_openSet
        
        # If the goal is reached, reconstruct and return the obtained path
        if current == goal:
            return reconstruct_path(cameFrom, current), closedSet

        openSet.remove(current)
        closedSet.append(current)
        
        # For each neighbor of the current node 
        for dx, dy, deltacost in movements:
            
            neighbor = (current[0]+dx, current[1]+dy)
            
            # If the node is not in the map, skip
            if (neighbor[0] >= occupancy_grid.shape[0]) or (neighbor[1] >= occupancy_grid.shape[1]) or (neighbor[0] < 0) or (neighbor[1] < 0):
                continue
            
            # If the node is occupied or has already been visited, skip
            if (occupancy_grid[neighbor[0], neighbor[1]]) or (neighbor in closedSet): 
                continue
            
            # Computation of the rotational cost by comparing the previous position, the current one and the next one
            if(not (current == start)):
                vector_prev = ([current[0] - (cameFrom[current])[0], current[1] - (cameFrom[current])[1]]) 
                vector_next = ([neighbor[0] - current[0], neighbor[1] - current[1]]) 
                angle = np.arccos(np.dot(vector_prev, vector_next) / (np.linalg.norm(vector_prev) * np.linalg.norm(vector_next)))
                rotation_cost = angle * ROTATION_COST
            else:
                rotation_cost = 0
                
            # The movement cost to the neighbor is the linear movement with deltacost and the rotation with rotational_cost
            # Tentative_gScore is the distance from start to the neighbor through current
            tentative_gScore = gScore[current] + deltacost + rotation_cost
            
            if neighbor not in openSet:
                openSet.append(neighbor)
            
            # If the path to the neighbor is better than any previous one, keep it
            if tentative_gScore < gScore[neighbor]:
                cameFrom[neighbor] = current
                gScore[neighbor] = tentative_gScore
                fScore[neighbor] = gScore[neighbor] + h[neighbor]

    # Open set is empty but goal was never reached
    print("No path found to goal")
    return [], closedSet

#--------------------------------------------------------------------------------#

def path_planning(robot_vector, goal_center, global_obstacle):
    """
    This function calls the other path planning functions to compute the cheapest path
    
    param :
        - robot_vector : current position of the Thymio robot  
        - goal_center : center position of the goal
        - global_obstacle : original grid with obstacle information
    
    return :
        - path_final : list of coordinates forming the cheapest path
    """
    
    # Generate the downsized obstacle grid
    occupancy_grid = create_downsized_grid(global_obstacle)
    
    # Size of the downscaled grid
    max_val_x = occupancy_grid.shape[0]
    max_val_y = occupancy_grid.shape[1]
    
    # Define the start and end goal while changing axis to use matrix coordinates
    start = (int(robot_vector[1]/reduction_coeff), int(robot_vector[0]/reduction_coeff))
    goal = (int(goal_center[1]/reduction_coeff), int(goal_center[0]/reduction_coeff))

    # List of all coordinates in the grid
    w,z = np.mgrid[0:max_val_x:1, 0:max_val_y:1]
    pos = np.empty(w.shape + (2,))
    pos[:, :, 0] = w; pos[:, :, 1] = z
    pos = np.reshape(pos, (w.shape[0]*w.shape[1], 2))
    coords = list([(int(w[0]), int(w[1])) for w in pos])
    
    # Define the heuristic, here = distance to goal ignoring obstacles
    h = np.linalg.norm(pos - goal, axis=-1)
    h = dict(zip(coords, h))

    # Run the A* algorithm    
    path, visitedNodes = A_Star(start, goal, h, coords, occupancy_grid)
    
    # Change axis to plot with same coordinates as the grid
    path = np.array(path).reshape(-1, 2).transpose()
    visitedNodes = np.array(visitedNodes).reshape(-1, 2).transpose()

    # Multiply the path coordinates by reduction_coeff to go back to the original grid scale
    path_final = path * reduction_coeff
    
    # Change axis to go back to the original coordinates
    path_final[[0,1]] = path_final[[1,0]] 
    
    # Replace the first and last node of the path to fit the true coordinates of the start and goal position
    path_final[0][0] = robot_vector[0]
    path_final[1][0] = robot_vector[1]
    path_final[0][-1] = goal_center[0]
    path_final[1][-1] = goal_center[1]
    
    # Displaying the map and the path information
    cmap = colors.ListedColormap(['white', 'red'])
    fig_astar, ax_astar = create_empty_plot(max_val_x, max_val_y)
    ax_astar.imshow(occupancy_grid, cmap=cmap)
    # Plot the best path found and the list of visited nodes
    ax_astar.scatter(visitedNodes[1], visitedNodes[0], marker="o", color = 'orange');
    ax_astar.plot(path[1], path[0], marker="o", color = 'blue');
    ax_astar.scatter(start[1], start[0], marker="o", color = 'green', s=200);
    ax_astar.scatter(goal[1], goal[0], marker="o", color = 'purple', s=200);
    
    return path_final

---
<a id="filtering"></a>
# Filtering


The localization of the Thymio robot is performed using a Kalman filter. This filtering method is well suited to estimating the position and orientation of a mobile robot from noisy or incomplete measurements. The design of the filter in this project is based on using the position ($x, y$) and orientation ($\theta$) provided by the camera as measurements. In addition, the speed of the robot, provided by the wheel speed sensors ($v_r, v_l$), is used as a prediction. In short, the Kalman filter merges a prediction of the system's future state with a measurement of that state to estimate position probabilistically.

<a id="state-space-model"></a>
## State-space model

### Prediction

To estimate the robot's future position, a state-space model needs to be developed: 

$$\hat{s}_{a\_priori}^{t+1} = A \cdot \hat{s}_{a\_posteriori}^{t} + B \cdot u^{t} + q^t$$

The prediction of the future state is referred to as $\hat{s}_{a\_priori}^{t+1}$, i.e. the a priori estimate at time t+1. Since the state of the system is defined by its position ($x, y$) and orientation ($\theta$), this gives: 

$$\hat{s}_{a\_priori}^{t+1} = \begin{pmatrix}
\hat{x}_{a\_priori}^{t+1} \\\\
\hat{y}_{a\_priori}^{t+1} \\\\
\hat{\theta}_{a\_priori}^{t+1}
\end{pmatrix}$$

The current state corresponds to the term $\hat{s}_{a\_posteriori}^{t}$, which is the a posteriori estimate at time t. In the same way as above, this gives:

$$\hat{s}_{a\_posteriori}^{t} = \begin{pmatrix}
\hat{x}_{a\_posteriori}^{t} \\\\
\hat{y}_{a\_posteriori}^{t} \\\\
\hat{\theta}_{a\_posteriori}^{t}
\end{pmatrix}$$

The system input at time t is represented by the vector $u^{t}$. This is made up of two terms: translational speed ($v$) and rotational speed ($\omega$). 

$$u^t = \begin{pmatrix}
v \\\\
\omega
\end{pmatrix} $$

These are defined on the basis of the speeds measured by the wheel speed sensors, i.e. the right ($v_r$) and left ($v_l$) speeds, and the spacing between the two wheels ($e$).

$$ v = \cfrac{v_r + v_l}{2} \qquad\qquad \omega = \cfrac{v_r-v_l}{e} $$ 

Matrix A characterizes the evolution of the system state, while matrix B describes the impact of the input on the future state. An odometry-based approach allows us to determine these two matrices by considering a very short time interval ($\delta t$). During this time interval, the robot rotates by $\delta \theta = \omega \cdot \delta t$. Knowing this, and referring to the diagram below, the following system of equations can be established: 

$$\begin{equation}
\begin{cases}
\hat{x}_{a\_priori}^{t+1} = \hat{x}_{a\_posteriori}^{t} + v \cdot \cos\left(\hat{\theta}_{a\_posteriori}^{t} + \delta \theta^t \right) \cdot \delta t \\
\hat{y}_{a\_priori}^{t+1} = \hat{y}_{a\_posteriori}^{t} + v \cdot \sin\left(\hat{\theta}_{a\_posteriori}^{t} + \delta \theta^t \right) \cdot \delta t \\
\hat{\theta}_{a\_priori}^{t+1} = \hat{\theta}_{a\_posteriori}^{t} + \omega \cdot \delta t
\end{cases}
\end{equation}$$

![state-space_model](Images/schema1.jpg)

The matrix form of this system therefore becomes:

$$\begin{equation}
A = \begin{bmatrix} 
1 & 0 & 0\\ 
0 & 1 & 0 \\ 
0 & 0 & 1
\end{bmatrix}
\qquad\qquad
B = \begin{bmatrix} 
\cos\left(\hat{\theta}_{a\_posteriori}^{t} + \delta \theta^t \right) \cdot \delta t & 0\\
\sin\left(\hat{\theta}_{a\_posteriori}^{t} + \delta \theta^t \right) \cdot \delta t & 0 \\
0 & \delta t 
\end{bmatrix}
\end{equation}$$

The final term $q^t$ of this state-space model represents the stochastic perturbation of the state with covariance matrix Q defined as follows:

$$
Q = \begin{bmatrix} 
q_1 & 0 & 0\\ 
0 & q_2 & 0 \\ 
0 & 0 & q_3
\end{bmatrix}
$$

These diagonal coefficients can be evaluated using an approach similar to that used in Exercise 8 of the MICRO-452 course. 

The experimentally estimated coefficients are as follows: q1=q2=4 (equivalent to a standard deviation of 2 pixels) and q3=0.03 (equivalent to a standard deviation of 10 degrees).


### Measurement

Having explored the prediction phase of the state-space model, attention now turns to the second essential part: updating the measurements. This stage aims to refine the predictions by integrating real information captured by the camera. The formula governing this step is :

$$ m^{t+1} = C \cdot s^{t+1} + r^{t+1}$$ 

Measurements taken at time t+1 are represented here by the term $m_{t+1}$. The data collected by the camera are therefore:

$$m^{t+1} = \begin{pmatrix}
x_{captured}^{t+1} \\\\
y_{captured}^{t+1} \\\\
\theta_{captured}^{t+1}
\end{pmatrix}$$

The robot's position ($x, y$) and orientation ($\theta$) measured by the camera are used directly as system outputs, without any transformation. The matrix C linking the measurements to the state is therefore defined as follows:

$$C = \begin{bmatrix} 
1 & 0 & 0\\ 
0 & 1 & 0 \\ 
0 & 0 & 1
\end{bmatrix}$$

The term $s^{t+1}$ simply represents the state of the system at time t+1:

$$s^{t+1} = \begin{pmatrix}
x^{t+1} \\\\
y^{t+1} \\\\
\theta^{t+1}
\end{pmatrix}$$

Finally, the last term $r^{t+1}$ of this equation represents noise on measurements with a covariance matrix R defined as follows:

$$
R = \begin{bmatrix} 
r_1 & 0 & 0\\ 
0 & r_2 & 0 \\ 
0 & 0 & r_3
\end{bmatrix}
$$

Based on the precision of the camera, the following coefficients have been estimated: r1=r2=1 (equivalent to a standard deviation of 1 pixel) and r3=0.01 (equivalent to a standard deviation of 6 degrees).

Note: When the camera's view is obstructed, estimation is only possible using the prediction model.


<a id="kalman-filter"></a>
## Kalman filter

In [ ]:
def filter_initialization():
    """
    Initialize the various vectors and matrices requiered for filtering
    
    robot_vector: position (x and y) and orientation (theta) taken from the camera vision
    """
    
    global s_prev_est_a_posteriori, P_prev_est_a_posteriori, A, B, u, C, Q, R

    ## Previous State A Posteriori Estimation Vector
    # Vector representing the estimated state of the system at the previous time step
    s_prev_est_a_posteriori = robot_vector
       
    ## Previous State A Posteriori Covariance Matrix
    # Matrix representing the estimated precision of the previous estimated state (same as R)
    P_prev_est_a_posteriori = np.array([[1, 0, 0], 
                                        [0, 1, 0], 
                                        [0, 0, 0.01]]) 
    
    ## State Matrix
    # Matrix defining how the system evolves from one time step to the next
    A = np.array([[1, 0, 0], 
                  [0, 1, 0], 
                  [0, 0, 1]])
        
    ## Input Matrix 
    # Matrix describing the impact of the input on the state
    B = np.array([[1, 0], 
                  [0, 1], 
                  [0, 0]]); 
        
    ## Input Vector
    # Vector representing control inputs applied to the system 
    u = np.array([0, 0])
    
    ## Output Matrix
    # Matrix linking measurements to state
    C = np.array([[1, 0, 0], 
                  [0, 1, 0], 
                  [0, 0, 1]])
        
    ## Process Noise Covariance Matrix
    # Covariance matrix representing uncertainty in system dynamics
    Q = np.array([[4, 0, 0], 
                  [0, 4, 0], 
                  [0, 0, 0.03]])
    
    ## Measurement Noise Covariance Matrix
    # Matrix representing uncertainty of camera measurements
    R = np.array([[1, 0, 0], 
                  [0, 1, 0], 
                  [0, 0, 0.01]])

#--------------------------------------------------------------------------------#
    
def update_input(v_l,v_r,update_time, direction_rotation):
    """
    Update the input vector and matrix
    
    v_l: robot x position deduced from the camera vision
    v_r: robot y position deduced from the camera vision
    update_time: robot theta orientation deduced from the camera vision
    """
    
    global B,u
    
    Thymio_to_mms = 0.349
    mm_to_px = 100/137
    
    # Average translational speed
    v = (v_r +v_l)/2 # Thymio speed (T)
    v = v * Thymio_to_mms * mm_to_px # Speed in px/s (T -> mm/s -> px/s)

    # Average rotational speed
    w = (v_r -v_l)*Thymio_to_mms/robot_diameter # Angular speed in rad/s
    
    if (direction_rotation == TURN_RIGHT):
        w = -w
    
    # Input vector
    u = np.array([v, w]) 
    
    # Angle variation
    delta_theta = w * update_time
    
    # Input matrix
    B = np.array([[np.cos(delta_theta + s_prev_est_a_posteriori[2])*update_time, 0],
                  [-np.sin(delta_theta + s_prev_est_a_posteriori[2])*update_time, 0], 
                  [0, update_time]]); 
    
#--------------------------------------------------------------------------------#

def kalman_filter(s_prev_est_a_posteriori, P_prev_est_a_posteriori):
    """
    Estimates the current state using the input sensor data and the previous state
    
    param s_prev_est_a_posteriori: previous state a posteriori estimation
    param P_prev_est_a_posteriori: previous state a posteriori covariance
    
    return s_est_a_posteriori: new a posteriori state estimation
    return P_est_a_posteriori: new a posteriori state covariance
    """
    
    ## Prediciton through the a priori estimate
    # estimated mean of the state
    s_est_a_priori = np.dot(A, s_prev_est_a_posteriori)+ np.dot(B, u);
    path_apriori.append(s_prev_est_a_posteriori)
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_prev_est_a_posteriori, A.T)) + Q
    
    ## Update         
    # m, C, and R for a posteriori estimate, depending on the detection of the camera
    if camera_on == True:
        m = robot_vector
        path_camera.append(m)
        # innovation / measurement residual
        i = m - np.dot(C, s_est_a_priori);
        # measurement prediction covariance
        S = np.dot(C, np.dot(P_est_a_priori, C.T)) + R;     
        # Kalman gain (tells how much the predictions should be corrected based on the measurements)
        K = np.dot(P_est_a_priori, np.dot(C.T, np.linalg.inv(S)));
        # a posteriori estimate
        s_est_a_posteriori = s_est_a_priori + np.dot(K,i);
        P_est_a_posteriori = P_est_a_priori - np.dot(K,np.dot(C, P_est_a_priori));
    else:
        K = 0 # Kalman gain is null because the camera can't deliver any data
        # a posteriori estimate
        s_est_a_posteriori = s_est_a_priori;
        P_est_a_posteriori = P_est_a_priori;
        
    return s_est_a_posteriori, P_est_a_posteriori

---
<a id="local-navigation"></a>

# Local navigation


Local navigation involves adjusting the robot's path to navigate around unexpected obstacles in its immediate vicinity. This often requires the robot to deviate from the optimal path to execute avoidance maneuvers. Thymio has to navigate past random obstacles, not captured in the vision and not factored into the global path planning. 

Once the obstacle is successfully navigated, Thymio can either return to the optimal path or calculate a new one. 

Thymio is equipped with five front horizontal proximity sensors for it to detect unknown objects that emerge in its path. Our primary objective is to help Thymio skillfully navigate around these local obstacles whilst enabling ample time for subsequent planning of an optimal path toward the ultimate goal.

**Input :**

    - Horizontal proximity sensor values; constantly updated

**Output :**

    - Command for controlling the robot's translational and rotational motion

**Challenges :**

    - Determining the colour of our physical obstacles have to be white, to not have the camera capture the obstacle in global vision.
    
    - Determining the shape of our obstacles have to be cylindrical for an optimal maneuver around it.
    
    - Determining the fixed distance to move forward and theta (angle) for Thymio's rotation was challenging to set without prior knowledge of the obstacle's shape. If the distance was too short or the angle too small, the robot will run into the obstacle. This tok us multiple tries to reach the optimal `distance_forward` and `theta`, as it was challenging to determine when an obstacle is considered cleared. Thus we had to finetune our parameters multiple times to ensure the success of our Thymio's obstacle avoidance. 
    
    - Determining the obstacle threshold for when Thymio should initiate the local avoidance, `obstThrh`.

### Parameters 

| Name                | Purpose                                                                           | Units | Global?|
| :------------------- | :------------------------------------------------------------------------------- |-------|-----|
| `obstThrh`      | High obstacle threshold for switching from global navigation to local navigation state| Int   | Y |
| `angle_turned` | Total angle turned since initial direction, before local avoidance                     | Radians| Y |
| `theta` | Angle to turn back, for Thymio to be aligned with initial direction before local avoidance    | Radians | N |
| `distance_forward`| Amount of distance to move forward  | Meters | Y |
| `diff_lr`| Calculated difference between the most left and most right sensors | Int | N |


In [7]:
async def local_avoidance(sensor_prox, theta, dist):
    """
    1a. Obstacle detected right in front: rotate left + go forward
    1b. Obstacle detected near sides: rotate + go forward
    2. Determine rotation left or right
    3. Wall following of the obstacle
    4. After obstacle clearance, turn back to initial direction
    """
    # Global variables
    global node, obstThrh
    
    # Calculate difference in Left & Right sensors
    diff_lr = sensor_prox[0] - sensor_prox[4]
    
    # Object detected right in front
    if sensor_prox[2] > obstThrh: 
        # Rotate to the left + go forward
        await turn(pi/4)  
        theta += pi/4
        print("Front obstacle, turning right")
        await move_forward(100*dist)
        await turn(-theta/2)
        print("Turning back to initial direction")
        await move_forward(dist*100)
        await turn(-theta/2)
        await turn(theta/2)
        return
    else:
        # Obstacle threshold
        if max(sensor_prox) > obstThrh:
            if (sensor_prox[0] + sensor_prox[1]) > (sensor_prox[3] + sensor_prox[4]):
                await turn(pi/24)
                theta += pi/24
                await move_forward(dist*2)
                print("Left obstacle, turning right")
                if (diff_lr) > 0:
                    await turn(pi/30)
                    theta += pi/30
                    await move_forward(dist)
                    print("Left wall following")
                else:
                    await move_forward(dist)
                    print("Not wall following")
            elif (sensor_prox[3] + sensor_prox[4]) > (sensor_prox[0] + sensor_prox[1]):
                await turn(-pi/24)
                theta -= pi/24
                await move_forward(dist*2)
                print("Right obstacle, turning left")
                if (diff_lr) < 0:
                    await turn(-pi/30)
                    theta -= pi/30
                    await move_forward(dist)
                    print("Right wall following")
                else:
                    await move_forward(dist)
                    print("Not wall following")
            else:
                return  # Do nothing
    
    # Obstacle cleared, move forward for a while before turning back to initial direction & move forward slightly more
    await move_forward(110*dist)
    await turn(-theta*dist*3.5)
    print("Turning back to initial direction")
    await move_forward(140*dist)
    await turn(theta*dist*3)
    await move_forward(dist)
    
    await motors_stop()
    return

# Local avoidance function with sensor values
async def la_function(sensor_prox): 
    await node.wait_for_variables()
    
    while sum(sensor_prox[i] > obstThrh for i in range(0, 5)) > 0: 
        sensor_prox = node["prox.horizontal"]
        print(list(sensor_prox))
        await local_avoidance(sensor_prox, angle_turned, distance_forward)
        await client.sleep(0.2)
        await motors_stop()
        print("Completed Local Avoidance!")
        return True
    return False


---
<a id="motion-control"></a>
# Motion control

<a id="connection-functions"></a>
## Connection functions

Initially, functions facilitating the establishment and termination of the connection with the robot have been implemented.

In [1]:
# Thymio connection
async def connect_Thymio():
    """
    Establish a connection with the Thymio if possible
    """
    global node, client
    try:
        client = ClientAsync()
        node = await asyncio.wait_for(client.wait_for_node(), timeout=2.0)
        await node.lock()
        print("Thymio connected")

    except asyncio.TimeoutError:
        print("Thymio not connected: Timeout while waiting for node.")
    except Exception as e:
        print(f"Thymio not connected: {str(e)}")
        
# Thymio disconnection
def disconnect_Thymio():
    """
    Enable to disconnect the Thymio
    """
    aw(node.stop())
    aw(node.unlock())
    print("Thymio disconnected")

<a id="motion-functions"></a>
## Motion functions

The robot’s movement from its starting position to the target is executed using the coordinates of the global path. To transition from one coordinate to the next, a specific method has been adopted. The robot begins by orienting itself towards the next point, taking into account its current orientation. Once the orientation is adjusted, the robot performs a straight-line movement to the next point. This step is repeated from position to position until the final destination is reached. The basic functions therefore include: activation and deactivation of motors, rotation by a defined angle, and linear movement over a defined distance.

In [ ]:
# Thymio set motors speeds  
async def set_speeds(left_speed, right_speed):
    """
    Enable to set the speed of the Thymio's wheels
    """
    global node
    v = {
        "motor.left.target":  [left_speed],
        "motor.right.target": [right_speed],
    }
    await node.set_variables(v)
    
# Thymio motors stop     
async def motors_stop():
    """
    Stop the Thymio
    """
    global node
    v = {
        "motor.left.target":  [0],
        "motor.right.target": [0],
    }
    await node.set_variables(v)    

# Conversion factors
Thymio_to_mms = 0.349
px_to_mm = 137/100
# Constants
ROTATION_SPEED = 100
TIME_FULL_TURN = (8800/1000)

# Thymio turns a specied angle
async def turn(angle):
    # Calculate the time needed to turn through the required angle
    rotation_time = (abs(angle) / (2*np.pi)) * TIME_FULL_TURN

    # Turn robot on itself
    # Check the sign of angle
    if np.sign(angle) > 0:
        # If angle is positive, turn left
        await set_speeds(-ROTATION_SPEED, ROTATION_SPEED)
        left_or_right = TURN_LEFT
    else:
        # If angle is negative, turn right
        await set_speeds(ROTATION_SPEED, -ROTATION_SPEED)
        left_or_right = TURN_RIGHT

    # Wait required time
    time.sleep(rotation_time)
    return rotation_time, left_or_right

# Constants
FORWARD_SPEED = 200 
TIME_PER_MM = 15.5/1000  # Time it takes for the robot to travel one meter at base speed

async def move_forward(distance_px):
    # Calculate the time needed to travel the requested distance
    
    distance_mm = distance_px * px_to_mm
    travel_time = (distance_mm) * TIME_PER_MM
    
    # Robot moves forward
    await set_speeds(FORWARD_SPEED, FORWARD_SPEED)

    # Wait for the necessary time
    time.sleep(travel_time)
    return travel_time

<a id="motion-control-function"></a>
## Motion control function

The management of one coordinate to another is entirely handled by the following function, which determines whether the next movement is a rotation or a translation, and establishes the angle or distance to be covered.

In [2]:
async def reach_next_node(next_node, mode, estimated_pos):

    # Vector between the estimated position and the next position in the global path
    vector_next_node = np.array([0,0])  
    vector_next_node[0] = path_final[0][next_node] - estimated_pos[0]
    vector_next_node[1] = path_final[1][next_node] - estimated_pos[1] 
    
    # Normalized angle between the estimated position and the next position in the global path
    gamma = -math.atan2(vector_next_node[1], vector_next_node[0]) - estimated_pos[2]
    gamma = (gamma + np.pi) % (2*np.pi) - np.pi
    
    # Distance separating the estimated position and the next position in the global path
    path_next_node = np.array([path_final[0][next_node], path_final[1][next_node]])
    path_current_node = np.array([estimated_pos[0], estimated_pos[1]])
    d = np.linalg.norm(path_next_node - path_current_node)
    
    if(not mode):
        if(abs(gamma) > ANGLE_THRESHOLD):
            time_r, left_or_right = await turn(gamma)
        else: 
            time_r = 0 
            left_or_right = 1  
        return time_r, left_or_right 
        
    if (mode):
        if( d > FORWARD_THRESHOLD):
            time_f = await move_forward(d)
            return time_f

---
<a id="project-structure"></a>
# Project structure


The project was realized by assembling various components. The camera, an essential element, provides the vision necessary for global navigation and filtering. It plays a crucial role in detecting the robot, the target, and fixed obstacles, thus allowing the calculation of the optimal path. It also provides position data ($x,y$) and the orientation ($\theta$) of the robot, which are indispensable for estimating its position when using the Kalman filter.

Global navigation generates the coordinates of the path to follow to reach the target. The Kalman filter, on the other hand, receives position and orientation data measured by the camera, as well as information related to motion control (wheel speeds and travel time). It outputs the best estimate of Thymio's real position.

Finally, the robot constantly interacts with local navigation and motion control to avoid local obstacles and move towards its destination. The motor commands are used to move the robot, while the frontal proximity sensors are used to detect local obstacles. 

![interactions](Images/diag1.jpg)

The functioning of the project is detailed below. Following initialization, a mapping is carried out, allowing the calculation of the global path. The program then enters a loop that continues until the robot reaches its destination. At each iteration of this loop, the robot checks whether it has been kidnnaped or if a mobile obstacle is in its path. If either of these situations occurs, the robot recalculates a global path (after avoiding any potential obstacle). If none of these situations occur, the robot performs the movement defined by the motion control. Once the movement is completed, the robot checks whether its current position corresponds to its final destination. It should be noted that the robot is capable of moving with or without the information provided by the camera.

![interactions](Images/diag2.jpg)

---
<a id="runnable-cells"></a>
# Runnable cells

<a id="functions-declaration"></a>
## Functions declaration

<a id="capture-image"></a>
## Capture image

<a id="main"></a>
## Main

In [ ]:
camera_on = False  
next_node = 1
GROUND_THRESHOLD = 500
calculate_global_path = False

path_final = path_planning(robot_vector, goal_center, obstacle_matrix)


await connect_Thymio()

filter_initialization()

while True:
    
    await node.wait_for_variables()
    sensor_prox = node["prox.horizontal"]
    await client.sleep(0.2)
    while sum(sensor_prox[i] > obstThrh for i in range(0, 5)) > 0: 
        calculate_global_path = await la_function(sensor_prox)
        
    await node.wait_for_variables()
    sensor_gr = node["prox.ground.reflected"]
    await client.sleep(0.2)
    while sum(sensor_gr[i] < GROUND_THRESHOLD for i in range(0, 2)): 
        calculate_global_path += 1
        await motors_stop()

    if(calculate_global_path):
        await motors_stop()
        path_final = path_planning(robot_vector, goal_center, obstacle_matrix)
        calculate_global_path = False

    
    
    

    
    # Rotation
    time_rotation, turn_left_or_right = await reach_next_node(next_node, ROTATION_MODE, s_prev_est_a_posteriori)
    update_input(-ROTATION_SPEED, ROTATION_SPEED, time_rotation, turn_left_or_right)
    s_prev_est_a_posteriori, P_prev_est_a_posteriori = kalman_filter(s_prev_est_a_posteriori, P_prev_est_a_posteriori)

    # Linear displacement
    forward_time = await reach_next_node(next_node, FORWARD_MODE , s_prev_est_a_posteriori)
    update_input(FORWARD_SPEED, FORWARD_SPEED, forward_time, 1)
    s_prev_est_a_posteriori, P_prev_est_a_posteriori = kalman_filter(s_prev_est_a_posteriori, P_prev_est_a_posteriori)
    
    next_node += 1
    
    if (next_node == path_final.shape[1]):
        break
    
await motors_stop()
disconnect_Thymio()
path_followed = np.array(path_followed)
#print(path_followed)
print("Thymio ready for the game!")

---
<a id="conclusion"></a>
# Conclusion

During this project, we have engineered and implemented various components, including vision, filtering, motion control, as well as global and local navigation. Our project is fully functional, and the Thymio is proficient in reaching its destination while adhering to the optimal path and circumventing obstacles through global navigation and local avoidance. We have successfully applied various techniques that we have learned over the semester in the *Basics of Mobile Robotics* course.

This project has been intriguing both technically and organizationally. The creation of a comprehensive project within a relatively short timeframe compelled us to make compromises. Nevertheless, the project operates efficiently across a wide array of tests.